In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

pd.set_option('display.max_columns', None)

pd.set_option('display.max_rows', 90)

In [ ]:
raw_data = pd.read_csv(r"/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
raw_data.tail(10)

In [ ]:
raw_data.select_dtypes("object").loc[:,raw_data.isna().sum() >0].columns

In [ ]:
data1 = raw_data.copy()

In [ ]:
for col in ["Alley","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2","GarageType","FireplaceQu","GarageFinish","GarageQual","GarageCond","PoolQC","Fence","MiscFeature"]:
    data1[col] = data1[col].fillna("None") # None has meaning

for col in ["MasVnrType","Electrical",]:
    data1[col] = data1[col].fillna(data1[col].mode()[0])

In [ ]:
data1.isnull().sum()

In [ ]:
null_num_cols = data1.select_dtypes("number").loc[:,raw_data.isna().sum() >0].columns

In [ ]:
for col in null_num_cols:
    data1[col] = data1[col].fillna(int(data1[col].mean()))

In [ ]:
plt.figure(figsize=(16, 12))
sns.heatmap(data1.select_dtypes("number").corr(), cmap='YlGnBu', annot=True, fmt='.1f', vmin=0);

In [ ]:
data1.isnull().sum().sum()

In [ ]:
num_cols = data1.select_dtypes("number").columns


In [ ]:
data2 = data1.copy()

In [ ]:
from pylab import subplot
a =1
plt.figure(figsize=(25,25))
for i in num_cols:
    a = a+1
    plt.subplot(8,5,a)
    ax = sns.histplot(data2[i])

In [ ]:
data2 = data2[data2["LotFrontage"]<150]

data2 = data2[data2["LotArea"]< 50000]

data2 = data2[data2["MasVnrArea"]< 500]

data2 = data2[data2["BsmtFinSF1"]< 2000]

data2 = data2[data2["BsmtFinSF2"]< 750]

data2 = data2[data2["TotalBsmtSF"]< 3500]

data2 = data2[data2["1stFlrSF"]< 3000]

data2 = data2[data2["2ndFlrSF"]< 1500]

data2 = data2[data2["LowQualFinSF"]< 100]

data2 = data2[data2["GrLivArea"]< 4000]

data2 = data2[data2["WoodDeckSF"]< 500]

data2 = data2[data2["OpenPorchSF"]< 300]

data2 = data2[data2["EnclosedPorch"]< 300]

data2 = data2[data2["3SsnPorch"]< 100]

data2 = data2[data2["ScreenPorch"]< 300]

#data2 = data2[data2["PoolArea"]< 100]

data2 = data2[data2["MiscVal"]< 2500]

data2 = data2[data2["SalePrice"]< 500000]

In [ ]:
data1.describe()

In [ ]:
from pylab import subplot
a =1
plt.figure(figsize=(25,25))
for i in num_cols:
    a = a+1
    plt.subplot(8,5,a)
    ax = sns.histplot(data2[i])

In [ ]:
a =1
plt.figure(figsize=(25,25))
for i in num_cols:
    a = a+1
    plt.subplot(8,5,a)
    ax = sns.scatterplot(x = data2[i],y = data2["SalePrice"])
    

In [ ]:
log_price = np.log(data2["SalePrice"])
data2["log_price"] = log_price

In [ ]:
a =1
plt.figure(figsize=(25,25))
for i in num_cols:
    a = a+1
    plt.subplot(8,5,a)
    ax = sns.scatterplot(x = data2[i],y =data2["log_price"])


In [ ]:
data3 = data2.copy()

In [ ]:
data3.drop(["Id","SalePrice"],axis=1,inplace=True)


In [ ]:
data3.select_dtypes("object").columns

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
var_mod = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition']

In [ ]:
for i in var_mod:
    data3[i]=le.fit_transform(data3[i])

In [ ]:
data3.head()

In [ ]:
target = data3["log_price"]
inputs = data3.drop(["log_price"],axis=1)

In [ ]:
inputs.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(inputs)

In [ ]:
scaled_inputs = scaler.transform(inputs)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(inputs,target,test_size=0.2,random_state=365)

In [ ]:
reg = LinearRegression()

reg.fit(x_train,y_train)

In [ ]:
reg.score(x_train,y_train)

In [ ]:
y_predicted = reg.predict(x_test)

In [ ]:
plt.scatter(y_test,y_predicted,alpha=0.2)
plt.xlabel("y_test")
plt.ylabel("y_predicted")

In [ ]:
df_pf = pd.DataFrame(np.exp(y_predicted),columns=["Predictions"]) # we take the exp for reversing the log 
y_test = y_test.reset_index(drop= True)
df_pf["Actual"] = np.exp(y_test)
df_pf.head(10).round(1)